# More Scalable GPs

In this notebook I will demonstrate a few implementations of some more advanced and scalable GPs. These methods use approximations via inducing inputs (a smart subset of the data to represent the entire dataset). This allows the GP algorithms to run in $\mathcal{O}(NM^2)$ instead of $\mathcal{O}(N^3)$ where $N$ are the number of samples and $M$ are the number of inducing inputs.

The algorithms to be used below are as follows:
* Sparse GP - FITC Approximation (**TODO**)
* Sparse GP - VFE Approximation (**Done**)

In [1]:
import numpy as np
from sklearn.datasets import make_regression
from sklearn.model_selection import train_test_split
from sklearn.base import clone
from sklearn.decomposition import PCA
# from sklearn.multioutput import MultiOutputRegressor
from sklearn.compose import TransformedTargetRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from sklearn.utils.estimator_checks import check_estimator
import time as time

%load_ext autoreload
%autoreload 2

In [2]:
# check_estimator(SVGP)

#### Import Module

So here you need to add the path to the models directory. There are automated ways to do this but just for a simple case, you can just manually add the directoy whenever you start the notebook. See below.

In [3]:
import sys

# Add the path to the models
sys.path.insert(0, '/media/disk/erc/papers/2019_ML_OCN/code/ml4ocean')
from src.models.utils import MultiTaskGP

# Import the GP Functions
from src.models.gpy import SGP, SVGP
import GPy

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [4]:
# Make Fake Dataset
X, y = make_regression(
    n_samples=20000, 
    n_features=20,    # Total Features
    n_informative=3,   # Informative Features 
    n_targets=10,
    bias=10,
    noise=0.8,
    random_state=123

)
train_size = 10000

# Training and Testing
xtrain, xtest, ytrain, ytest = train_test_split(
    X, y, train_size=train_size, random_state=123
)

xtrain.shape, ytrain.shape

((10000, 20), (10000, 10))

### Sparse Variational GP - MultiOutput (Shared Kernel, Shared Outputs)

In [ ]:
# Define Kernel Function
input_dimensions = X.shape[1]
kernel = GPy.kern.RBF(
    input_dim=input_dimensions, 
    ARD=False
)

# define GP model
n_inducing = 25
gp_model = SVGP(
    kernel=kernel,
    n_inducing=n_inducing,
    max_iters=300, 
    optimizer='lbfgs',
    verbose=True,
    n_restarts=10
)


# train GP Model
# print(xtrain, ytrain)
t0 = time.time()
gp_model.fit(xtrain, ytrain)
t1 = time.time() - t0

# Predictions
ypred, ystd = gp_model.predict(xtest, return_std=True)

print(
#     f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f"Training Time: {t1:.3} seconds"
)

In [ ]:
ypred.shape, ystd.shape

In [ ]:
gp_model.display_model()

In [ ]:
# Get Stats
mae = mean_absolute_error(ypred, ytest)
mse = mean_squared_error(ypred, ytest)
rmse = np.sqrt(mse)
r2 = r2_score(ypred, ytest)

print(
    f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f" \nTime: {t1:.3} seconds"
)

### SVGP - Multi-Task

In [11]:
# Define Kernel Function
input_dimensions = X.shape[1]
kernel = GPy.kern.RBF(
    input_dim=input_dimensions, 
    ARD=False
)

# define GP model
n_inducing = 100
gp_model = SVGP(
    kernel=kernel,
    n_inducing=n_inducing,
    max_iters=300, 
    optimizer='lbfgs',
    verbose=False,
    n_restarts=0
)

# Define Multioutput function
gp_model_multi = MultiTaskGP(
    gp_model, 
    n_jobs=1,              # Number of cores to use to parallelize the training
)

# train GP Model
t0 = time.time()
gp_model_multi.fit(xtrain, ytrain)
t1 = time.time() - t0

print(
    f"Training \nTime: {t1:.3} seconds"
)

KeyboardInterrupt caught, calling on_optimization_end() to round things up


KeyboardInterrupt: 

In [ ]:
# Predict with test set
t0 = time.time()
ypred1, ystd = gp_model_multi.predict(xtest, return_std=True)
print(ypred.shape, ystd.shape)
ypred2 = gp_model_multi.predict(xtest)
np.testing.assert_array_equal(ypred1, ypred2)
t1 = time.time() - t0
# Get Stats
mae = mean_absolute_error(ypred2, ytest)
mse = mean_squared_error(ypred2, ytest)
rmse = np.sqrt(mse)
r2 = r2_score(ypred2, ytest)

print(
    f"GP Model:\n"
    f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f" \nTime: {t1:.3} seconds"
)

In [ ]:
# Get Stats
mae = mean_absolute_error(ypred, ytest)
mse = mean_squared_error(ypred, ytest)
rmse = np.sqrt(mse)
r2 = r2_score(ypred, ytest)

print(
    f"MAE: {mae:.3f}\nMSE: {mse:.3f}\nRMSE: {rmse:.3f}\nR2: {r2:.3f}" 
    f" \nTime: {t1:.3} seconds"
)

In [ ]:
# Inverse transform predictions
# ypred = pca_model.inverse_transform(ypred_red)

In [ ]:
# Define target transformer
pca_model = PCA(n_components=3)

# Transform Targes
ytrain_red = pca_model.fit_transform(ytrain)